In [1]:
import fast_rl.core.Interpreter
import fast_rl.core.Learner 
import fast_rl.agents.DQN 
from fast_rl.agents.DQN import DQN, FixedTargetDQN, DoubleDQN, DuelingDQN, DoubleDuelingDQN
from fast_rl.core.Interpreter import AgentInterpretationAlpha
from fast_rl.core.Learner import AgentLearner
from fast_rl.core.MarkovDecisionProcess import MDPDataBunch
from fast_rl.core.agent_core import PriorityExperienceReplay, ExperienceReplay
from fast_rl.core.MarkovDecisionProcess import MDPDataBunch, FEED_TYPE_IMAGE, FEED_TYPE_STATE
import sys
import importlib

In [2]:
importlib.reload(fast_rl.core.Learner)
importlib.reload(fast_rl.agents)

data = MDPDataBunch.from_env('maze-random-5x5-v0', render='human', max_steps=1000, add_valid=False, 
                            feed_type=FEED_TYPE_STATE)
model = FixedTargetDQN(data, batch_size=128, max_episodes=10000, lr=0.001, copy_over_frequency=3,
                       memory=ExperienceReplay(10000), discount=0.99)
learn = AgentLearner(data, model)

learn.fit(40)
interp = AgentInterpretationAlpha(learn, base_chart_size=(20, 10))
interp.plot_heatmapped_episode(-1, return_heat_maps=False)


pygame 2.0.0.dev3 (SDL 2.0.9, python 3.6.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 

In [ ]:
for i in range(4):
    interp.plot_heatmapped_episode(-1, action_index=i, return_heat_maps=False)

In [ ]:
# learn.fit(10)

In [ ]:
for i in range(4):
    interp.plot_heatmapped_episode(-1, action_index=i, return_heat_maps=False)

In [ ]:
# data = MDPDataBunch.from_env('CartPole-v1', render='human', max_steps=100, add_valid=False)
# model = FixedTargetDQN(data, batch_size=128, max_episodes=1000, lr=0.0001, copy_over_frequency=3,
#                        memory=ExperienceReplay(100), discount=0.99)
# learn = AgentLearner(data, model)

# learn.fit(450)

Ok the rewards should not be positive....

In [ ]:
interp = AgentInterpretationAlpha(learn, base_chart_size=(20, 10))
interp.plot_agent_accuracy_density()

In [ ]:
importlib.reload(fast_rl.core.Interpreter)
interp.plot_q_density()

In [ ]:
# learn.fit(1)

In [ ]:
# interp = AgentInterpretationAlpha(learn, base_chart_size=(20, 10))
interp.plot_heatmapped_episode(-1, return_heat_maps=False)

# interp = AgentInterpretationAlpha(learn, base_chart_size=(20, 10))
# interp.plot_memory_samples(batch_size=1000, key='action')
interp.plot_q_density()

In [ ]:
interp.plot_memory_samples(batch_size=100, key='action')

In [ ]:
self = learn.model
import torch
import numpy as np

In [ ]:
sampled = self.memory.sample(self.batch_size)
with torch.no_grad():
    r = torch.from_numpy(np.array([item.reward for item in sampled])).float()
    s_prime = torch.from_numpy(np.array([item.result_state for item in sampled])).float()
    s = torch.from_numpy(np.array([item.current_state for item in sampled])).float()
    a = torch.from_numpy(np.array([item.actions for item in sampled])).long()
print(s)

In [ ]:
s

In [ ]:
y_hat = self.action_model(s).gather(1, a)

In [ ]:
y_hat

In [ ]:
list(self.target_net.parameters())

In [ ]:
0.99 * self.target_net(s_prime).max(axis=1)[0].unsqueeze(1) + r.expand_as(y_hat)

In [ ]:
 r.expand_as(y_hat)

In [ ]:
list(AgentLearner(data, FixedTargetDQN(data)).model.parameters())